*Exercice réalisé par Michel Nguyen.*

**AVANT D'EXECUTER CE NOTEBOOK, IL FAUT AU PRÉALABLE EXECUTER LE NOTEBOOK PRÉCÉDENT**

Ce notebook a pour but de réaliser un API sur la base du notebook précédent. On souhaite créer une interface web telle qu'elle puisse prédire les équipements principaux à partir d'une liste exhaustive d'équipements.

Pour cela, nous utiliserons la librairie flask.

In [1]:
# création d'API avec flask
from flask import Flask, request, render_template

# analyse des données
import pandas as pd

# sauvegarde du modèle
import pickle

# modèles de prédictions
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# définir l'application
app = Flask("__name__")

q = ""

In [3]:
# définir la page home en utlisant un document html
# ce document html doit contenir une case où on insère la liste exhaustive d'équipements ; un 
# bouton pour envoyer une requête ; ainsi qu'une case où apparait la prédiction
@app.route("/")
def loadPage():
    return render_template('home.html', query="")

In [4]:
# définir la prédiction à partir de la liste exhaustive envoyée par l'utilisateur
# on reprend presque tous les éléménts utiles du notebook précédent pour la construction du modèle de prédiction
@app.route("/", methods=['GET','POST'])
def carPrediction():
    df = pd.read_csv('cars_data.csv', sep=";")
    df = df[:99]
    
    df['main_equipments'] = df['main_equipments'].apply(lambda x: str(x).split(", "))
    X = df.iloc[:, 5].values
    y = df.iloc[:, -1].values
    X = X[:99]
    y = y[:99]
    
    X_train, X_test, y_train_text, y_test_text = train_test_split(X,y, test_size = 0.2,random_state = 0)

    Y_train_text = []
    for i in range(0,len(y_train_text)):
        Y_train_text.append(", ".join(y_train_text[i]))
    Y_train_text

    for i in range(0,len(Y_train_text)):
        Y_train_text[i] = Y_train_text[i].split(', ')

    Y_test_text = []
    for i in range(0,len(y_test_text)):
        Y_test_text.append(", ".join(y_test_text[i]))

    for i in range(0,len(Y_test_text)):
        Y_test_text[i] = Y_test_text[i].split(', ')
    
    mlb = preprocessing.MultiLabelBinarizer(classes=('4 roues motrices (4x4)', '7 places', 'Android Auto',
     'Apple Car Play',
     'Bluetooth',
     'Caméra de recul',
     'Climatisation',
     'GPS',
     'Intérieur cuir',
     'Jantes alliage',
     'Radar de recul',
     'Régulateur de vitesse',
     'Toit ouvrant/panoramique'))
    Y = mlb.fit_transform(Y_train_text)
    Y_test = mlb.fit_transform(Y_test_text)

    # utilisation du fichier 'classifier.sav'
    classifier = pickle.load(open('classifier.sav', 'rb'))
    
    # 'query1' est la liste exhaustive envoyée par l'utilisateur
    newCar = [request.form['query1']]
    # on soumet query1 au modèle de prédiction
    prediction = classifier.predict(newCar)
    all_labels = mlb.inverse_transform(prediction)
    # 'o1' est la prédiction selon le modèle de prédiction
    o1 = []
    for item, labels in zip(newCar, all_labels):
        o1.append(labels)
        
    return render_template('home.html', output1=o1, query1 = request.form['query1'])

In [ ]:
app.run(port=5000)

 * Serving Flask app "__name__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jan/2022 11:53:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2022 11:53:59] "POST / HTTP/1.1" 200 -
